<a href="https://colab.research.google.com/github/lucasreis95/world-surf-league-data/blob/main/notebooks/04_silver_ranking_yearly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libs
import pandas as pd
import pandas_gbq
import numpy as np
import re

In [ ]:
# read df from gbq
df_raw = pandas_gbq.read_gbq(
                             query_or_table = 'wsl-data-397017.01_bronze.wsl_ranking_scrap'
                             )

Downloading: 100%|██████████|


In [ ]:
df_raw.columns

Index(['index', 'rank', 'unnamed__1_level_1', 'unnamed__2_level_1', 'name',
       '1_artboard', '2_artboard', '3_artboard', '4_artboard', '5_artboard',
       '6_artboard', '7_artboard', '8_artboard', '9_artboard', '10_artboard',
       'total_points', 'year', 'processing_date', '11_artboard',
       'regular_season_total_points'],
      dtype='object')

In [ ]:
df = df_raw.copy()
# set empty values as np.nan
df = df.replace('None', np.nan)
df = df.replace('-', np.nan, regex=True)
# drop columns
df = df.drop(columns = ['unnamed__1_level_1', 'unnamed__2_level_1', 'processing_date'])
#
df['name'] = df['name'].str.replace('United States','United_states')\
                       .str.replace('South Africa','South_africa')\
                       .str.replace('New Zealand','New_zealand')\
                       .str.replace('French Polynesia','French_polynesia')\
                       .str.replace('Costa Rica','Costa_rica')

# add space before last capital letter, that means before surfer country
df['name'] = df['name'].str.replace( r"([A-Z])", r" \1", regex=True).str.strip()
# split name and country by last space
df[['name','athlete_country']] = df['name'].str.rsplit(expand=True, n=1)
# some surfers have () after their names, that means something happened, let's remove those ()
# ex: Julian Wilson (W D N)	on 2021 rank)
df['name'] = df['name'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "",str(x)))
# remove * from numbers (need to use '/' because '*' is used in regex)
df = df.replace({'\*':'', ',':''}, regex = True)
# filter out rows containing cutoff
df = df[~df['rank'].isin(['Final 5 Cutoff', 'Cut Line', 'WSL Final 5 Cutoff', 'Mid-Season Cut Line'])]
# lower case and strip names and countries
df['athlete_country'] = df['athlete_country'].str.lower()
df['athlete_country'] = df['athlete_country'].str.replace('_', ' ')
df['name'] = df['name'].str.lower()
df['name']= df['name'].replace(r'\s+', ' ', regex=True)
# rename columns
df.columns = df.columns.str.replace('artboard', 'event_points_earned')
df = df.rename(columns = {
                          'rank':'rank_position',
                          'name':'athlete_name',
                          'year':'season_year'
                          })

# change columns types
df['season_year'] = df['season_year'].astype(int)
df['rank_position'] = df['rank_position'].astype(float)
df['1_event_points_earned'] = df['1_event_points_earned'].astype(float)
df['2_event_points_earned'] = df['2_event_points_earned'].astype(float)
df['3_event_points_earned'] = df['3_event_points_earned'].astype(float)
df['4_event_points_earned'] = df['4_event_points_earned'].astype(float)
df['5_event_points_earned'] = df['5_event_points_earned'].astype(float)
df['6_event_points_earned'] = df['6_event_points_earned'].astype(float)
df['7_event_points_earned'] = df['7_event_points_earned'].astype(float)
df['8_event_points_earned'] = df['8_event_points_earned'].astype(float)
df['9_event_points_earned'] = df['9_event_points_earned'].astype(float)
df['10_event_points_earned'] = df['10_event_points_earned'].astype(float)
df['11_event_points_earned'] = df['11_event_points_earned'].astype(float)
df['total_points'] = df['total_points'].astype(float)
df['total_points'] = df['total_points'].astype(float)
df['regular_season_total_points'] = df['regular_season_total_points'].astype(float)

# reorder columns
df = df[[
         'season_year',
         'athlete_name',
         'athlete_country',
         'rank_position',
         'regular_season_total_points',
         'total_points',
         '1_event_points_earned',
         '2_event_points_earned',
         '3_event_points_earned',
         '4_event_points_earned',
         '5_event_points_earned',
         '6_event_points_earned',
         '7_event_points_earned',
         '8_event_points_earned',
         '9_event_points_earned',
         '10_event_points_earned',
         '11_event_points_earned'
        ]]

In [ ]:
df.head()

,season_year,athlete_name,athlete_country,rank_position,regular_season_total_points,total_points,1_event_points_earned,2_event_points_earned,3_event_points_earned,4_event_points_earned,5_event_points_earned,6_event_points_earned,7_event_points_earned,8_event_points_earned,9_event_points_earned,10_event_points_earned,11_event_points_earned
0,2010,dean morrison,australia,35.0,NaN,8500.0,1750.0,500.0,500.0,1750.0,1750.0,NaN,NaN,1750.0,NaN,500.0,NaN
1,2010,neco padaratz,brazil,37.0,NaN,6250.0,500.0,1750.0,1750.0,1750.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,kekoa bacalso,hawaii,37.0,NaN,6250.0,1750.0,500.0,1750.0,1750.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,ben dunn,australia,37.0,NaN,6250.0,500.0,500.0,1750.0,1750.0,1750.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,tanner gudauskas,united states,37.0,NaN,6250.0,500.0,1750.0,1750.0,1750.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# write raw table in big query
pandas_gbq.to_gbq(
                  dataframe = df,
                  destination_table = 'wsl-data-397017.02_silver.wsl_ranking_yearly',
                  project_id = 'wsl-data-397017',
                  if_exists = 'replace'
                  )

100%|██████████| 1/1 [00:00<00:00, 788.11it/s]
